# template_webscraping

## Load Libraries

In [7]:
# Selenium
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
# Reqeusts
import requests
# Other tools
from bs4 import BeautifulSoup
from datetime import datetime
from datetime import timedelta
from datetime import date
from datetime import datetime
from datetime import timedelta
from datetime import date
import csv
import pandas as pd
import time
import json
import fnmatch
import os
import tabula
from tabula.io import read_pdf
from tqdm import tqdm


## Functions Using Requests

In [3]:
def requests_get_item(url, item):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"}
    if item == "html":       
        page = requests.get(url, headers = headers)
        soup = BeautifulSoup(page.text, 'html.parser')
        return(soup)
    
    elif item == "pdf":
        page = requests.get(url, headers = headers, stream=True)
        return(page)
    else:
        print("Valid Item Not Selected")

def download_pdfs(ls_pdf_urls, download_path, file_name):
    counter = 0 
    for pdf_url in ls_pdf_urls:
        counter = counter +1
        g = requests_get_item(pdf_url, "pdf")
        with open(f'{download_path}{file_name}_{counter}.pdf', 'wb') as sav:
            for chunk in g.iter_content(chunk_size=1000000):
                sav.write(chunk)
        print(f"download number: {counter}")
                
                
def convert_pdf_to_csv(pdf_directory, csv_directory):
    directory = fr'{pdf_directory}'
    directory_output = fr'{csv_directory}'
    count = 0
    for file in os.listdir(directory):        
        if file.endswith(".pdf"):
            count = count + 1 
            print(f'{directory}{file}: Conversion {count}')
            tabula.convert_into(f'{directory}{file}', f'{directory_output}{count}.csv', output_format="csv", pages='all')
    
    
    
    

In [4]:
def write_json_data(data_out, database_name, file_path):
        #writing data to json
        data_out = data_out
        database_name = database_name
        file = f'{file_path}/{database_name}'
        with open(file, 'w') as outfile:
            json.dump(data_out, outfile)
            
def open_json_data(file_path):    
    with open(file_path) as f:
        data = json.load(f)
    return data

In [5]:
# date must be in following format: 2021-08-12
def get_date(start_date, end_date):
    date_list = []
    # end_date as current data
    if end_date == "today":
        end_date = date.today()
    
    # add 60 days to start date, save that as first el in list
    start_date = datetime.strptime(start_date, '%Y-%m-%d')
    start_date = start_date + timedelta(days=60)
    start_date = (start_date.date())
    
    date_list.append(str(start_date))
    while (start_date + timedelta(days=60)) < end_date:
        start_date = start_date + timedelta(days=120)
        string_start_date = str(start_date)
        
        if (start_date + timedelta(days=60)) > end_date: 
            start_date = end_date
        date_list.append(str(start_date))
    
        
    return(date_list)
        # while start + 60 is less than today, keep adding a new date that's +120 days
        # when it's greater thant today's date, break out of the lop
    
    # next date will be 120 days after the date we just returned
    # while loop intil we hit a date that's within 60 days of today's end_date
    
dates = get_date('2001-01-03', 'today')
print(dates)

['2001-03-04', '2001-07-02', '2001-10-30', '2002-02-27', '2002-06-27', '2002-10-25', '2003-02-22', '2003-06-22', '2003-10-20', '2004-02-17', '2004-06-16', '2004-10-14', '2005-02-11', '2005-06-11', '2005-10-09', '2006-02-06', '2006-06-06', '2006-10-04', '2007-02-01', '2007-06-01', '2007-09-29', '2008-01-27', '2008-05-26', '2008-09-23', '2009-01-21', '2009-05-21', '2009-09-18', '2010-01-16', '2010-05-16', '2010-09-13', '2011-01-11', '2011-05-11', '2011-09-08', '2012-01-06', '2012-05-05', '2012-09-02', '2012-12-31', '2013-04-30', '2013-08-28', '2013-12-26', '2014-04-25', '2014-08-23', '2014-12-21', '2015-04-20', '2015-08-18', '2015-12-16', '2016-04-14', '2016-08-12', '2016-12-10', '2017-04-09', '2017-08-07', '2017-12-05', '2018-04-04', '2018-08-02', '2018-11-30', '2019-03-30', '2019-07-28', '2019-11-25', '2020-03-24', '2020-07-22', '2020-11-19', '2021-03-19', '2021-07-17', '2021-11-28']


In [9]:
### generate urls -
# url = https://www.salisbury.edu/_services/police/crimebeat?date=07%2F12%2F2017


base_url = 'https://www.salisbury.edu/_services/police/crimebeat?date='
results = []
counter = 0
counter2 = 0
for date in tqdm(dates):
    split_date = date.split('-')
    
    year = (split_date[0])
    month = (split_date[1])
    day = (split_date[2])     
    url = (f'{base_url}{month}%2F{day}%2F{year}')         
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')         
    result = soup.text
    file = f'../data/handmade/salisbury/jsons_final/salisbury_{year}_{month}_{day}.json'
    with open(file, 'w') as outfile:
        json.dump(result, outfile)


100%|██████████████████████████████████████████████████████████████████████████████████| 64/64 [00:30<00:00,  2.07it/s]
